In [1]:
# Goal: implement eps-FOCuS algorithm that works for both mu>0 and mu<0, where the sign isn't known.  
# Do this with epsilon-greedy approach that maximizes absolute value of sample mean
# Experiment with changepoint estimate location over time and see if it's deviation is bounded
# Experiment with whether stream 1 is the stream used to estimate changepoint location always after a certain constant

In [30]:
import numpy as np
from scipy.stats import norm
import random

nu = int(1e4)
T = int(1e6)
M = 10
# Assume changepoint is in stream 1
data = np.zeros((M,T))
mu0 = 0
var = 1
mu1 = -0.1
data[0][:nu] = norm.rvs(size=nu)
data[0][nu:] = norm.rvs(loc=mu1,size=T-nu)
for i in range(1,M):
    data[i] = norm.rvs(loc=mu0,size=T)

In [28]:
epsilon = 0.1
threshold = 100

In [26]:
def changepoint_estimator(quadratics_positive,quadratics_negative,S,n,M):
    # n records the number of samples taken for each stream, equal to the last quadratic's local time step entered
    best_glr = 0
    # randomized selection if no stream can exceed a glr of 0
    best_stream = random.randint(0,M)
    best_changepoint = 0
    # Find stream and changepoint containing best GLR statistic
    for m in range(M):
        # iterate through stream m's quadratics
        # if no samples then keep going
        if n[m] == 0:
            continue
        else:
            for quadratic in quadratics_positive[m] + quadratics_negative[m]:
                if n[m]-quadratic[0]==0: 
                    continue
                elif (S[m]-quadratic[1])**2/(2*(n[m]-quadratic[0]))>best_glr:
                    best_glr = (S[m]-quadratic[1])**2/(2*(n[m]-quadratic[0]))
                    best_changepoint = quadratic[3]
                    best_stream = m
    return best_changepoint, best_stream, best_glr

In [51]:
import sys
from scipy.stats import bernoulli

epsilon = 0.1
threshold = 200
nu = int(1e4)
T = int(1e6)
M = 10
# Assume changepoint is in stream 1
data = np.zeros((M,T))
mu0 = 0
var = 1
mu1 = -0.1
data[0][:nu] = norm.rvs(size=nu)
data[0][nu:] = norm.rvs(loc=mu1,size=T-nu)
for i in range(1,M):
    data[i] = norm.rvs(loc=mu0,size=T)
# Implementation of multi-stream FOCuS for any mu
# Each stream gets its own set of quadratics
quadratics_positive = [[(0,0,0,0)] for m in range(M)]
quadratics_negative = [[(0,0,0,0)] for m in range(M)]
# tau, s, l, g
# g corresponds to global time step, tau corresponds to local time step (number of samples taken, N(g))
# Again each stream gets its own set of coefficients so there is M-length array for S
S = [0 for m in range(M)]
# And M-length array for number of samples at current time step
n = [0 for m in range(M)] 

# record location of changepoint estimates
changepoint_history = []
# record stream selected at each time step
stream_mle_history = []

# history after each time step
history_count = np.zeros((M,T))
history_reward = np.zeros((M,T))
history_sample_means = np.zeros((M,T))
# Need to make sure history is sub-linear in T
# For now I can use a history vector but I should work on an implementation
# One way is I can put a state vector in each changepoint location
# This has M^2 log T memory complexity but this is better than T complexity if M is fixed
# So we assume changepoint_location is the actual changepoint.  After that we cut off the data
# Remember the changepoint is 1-indexed but Python is zero-indexed so subtracting 

for t in range(T):
    # First perform stream selection with epsilon-greedy
    exploration = bernoulli.rvs(epsilon)
    if exploration:
        a_t = np.random.randint(M)
    else:
        changepoint_location, stream_estimate, _ = changepoint_estimator(quadratics_positive,quadratics_negative,S,n,M)
        # Check if there exists any stream which do not have any post-change samples 
        M_0 = np.where((history_count[:,t-1]-history_count[:,changepoint_location])==0)[0]
        if len(M_0) == 0:
            sample_means = (history_reward[:,t-1]-history_reward[:,changepoint_location])/(history_count[:,t-1]-history_count[:,changepoint_location])
            a_t = np.argmax(np.abs(sample_means))
            history_sample_means[:,t]=sample_means
        else:
            a_t = np.random.choice(M_0)
    x_t = data[a_t,t]
    
    n[a_t] = n[a_t] + 1
    for m in range(M):
        history_count[m,t] = n[m]
    S[a_t] = S[a_t] + x_t
    for m in range(M):
        history_reward[m,t] = S[m]
    k = len(quadratics_positive[a_t])
    quadratic_add = [n[a_t],S[a_t],np.inf,t+1]
    i = k
    while (2*(quadratic_add[1]-quadratics_positive[a_t][i-1][1])-(quadratic_add[0]-quadratics_positive[a_t][i-1][0])*quadratics_positive[a_t][i-1][2])<=0 and i>=1:
        i = i-1
    quadratic_add[2] = max(0,2*(quadratic_add[0]-quadratics_positive[a_t][i-1][0])/(quadratic_add[2]-quadratics_positive[a_t][i-1][2]))
    quadratics_positive[a_t] = quadratics_positive[a_t][:i].copy()
    quadratics_positive[a_t].append(tuple(quadratic_add))

    # Now negative update
    k = len(quadratics_negative[a_t])
    quadratic_add = [n[a_t],S[a_t],-np.inf,t+1]
    i = k
    while (2*(quadratic_add[1]-quadratics_negative[a_t][i-1][1])-(quadratic_add[0]-quadratics_negative[a_t][i-1][0])*quadratics_negative[a_t][i-1][2])>=0 and i>=1:
        i = i-1
    quadratic_add[2] = min(0,2*(quadratic_add[0]-quadratics_negative[a_t][i-1][0])/(quadratic_add[2]-quadratics_negative[a_t][i-1][2]))
    quadratics_negative[a_t] = quadratics_negative[a_t][:i].copy()
    quadratics_negative[a_t].append(tuple(quadratic_add))
    
    changepoint_location, stream_estimate, best_glr = changepoint_estimator(quadratics_positive,quadratics_negative,S,n,M)
    changepoint_history.append(changepoint_location)
    stream_mle_history.append(stream_estimate)
    if best_glr>=threshold:
        print(f'stopping time: {t+1}')
        print(best_glr)
        break

stopping time: 68228
200.12809140515796


In [56]:
changepoint_history[-100:]

[9392,
 9392,
 9392,
 9392,
 9392,
 9392,
 9392,
 9392,
 9392,
 9392,
 9392,
 9392,
 9392,
 9392,
 9392,
 9392,
 9392,
 9392,
 9392,
 9392,
 9392,
 9392,
 9392,
 9392,
 9392,
 9392,
 9392,
 9392,
 9392,
 9392,
 9392,
 9392,
 9392,
 9392,
 9392,
 9392,
 9392,
 9392,
 9392,
 9392,
 9392,
 9392,
 9392,
 9392,
 9392,
 9392,
 9392,
 9392,
 9392,
 9392,
 9392,
 9392,
 9392,
 9392,
 9392,
 9392,
 9392,
 9392,
 9392,
 9392,
 9392,
 9392,
 9392,
 9392,
 9392,
 9392,
 9392,
 9392,
 9392,
 9392,
 9392,
 9392,
 9392,
 9392,
 9392,
 9392,
 9392,
 9392,
 9392,
 9392,
 9392,
 9392,
 9392,
 9392,
 9392,
 9392,
 9392,
 9392,
 9392,
 9392,
 9392,
 9392,
 9392,
 9392,
 9392,
 9392,
 9392,
 9392,
 9392,
 9392]

In [52]:
sample_means

array([-0.09527672, -0.02267894, -0.04166375,  0.01262587,  0.04536723,
       -0.05157093, -0.01935794, -0.02862608, -0.03290522, -0.03189808])

In [53]:
import sys
from scipy.stats import bernoulli

epsilon = 0.1
threshold = 200
nu = int(1e4)
T = int(1e6)
M = 1
# Assume changepoint is in stream 1
data = np.zeros((M,T))
mu0 = 0
var = 1
mu1 = -0.1
data[0][:nu] = norm.rvs(size=nu)
data[0][nu:] = norm.rvs(loc=mu1,size=T-nu)
for i in range(1,M):
    data[i] = norm.rvs(loc=mu0,size=T)
# Implementation of multi-stream FOCuS for any mu
# Each stream gets its own set of quadratics
quadratics_positive = [[(0,0,0,0)] for m in range(M)]
quadratics_negative = [[(0,0,0,0)] for m in range(M)]
# tau, s, l, g
# g corresponds to global time step, tau corresponds to local time step (number of samples taken, N(g))
# Again each stream gets its own set of coefficients so there is M-length array for S
S = [0 for m in range(M)]
# And M-length array for number of samples at current time step
n = [0 for m in range(M)] 

# record location of changepoint estimates
changepoint_history = []
# record stream selected at each time step
stream_mle_history = []

# history after each time step
history_count = np.zeros((M,T))
history_reward = np.zeros((M,T))
history_sample_means = np.zeros((M,T))
# Need to make sure history is sub-linear in T
# For now I can use a history vector but I should work on an implementation
# One way is I can put a state vector in each changepoint location
# This has M^2 log T memory complexity but this is better than T complexity if M is fixed
# So we assume changepoint_location is the actual changepoint.  After that we cut off the data
# Remember the changepoint is 1-indexed but Python is zero-indexed so subtracting 

for t in range(T):
    # First perform stream selection with epsilon-greedy
    exploration = bernoulli.rvs(epsilon)
    if exploration:
        a_t = np.random.randint(M)
    else:
        changepoint_location, stream_estimate, _ = changepoint_estimator(quadratics_positive,quadratics_negative,S,n,M)
        # Check if there exists any stream which do not have any post-change samples 
        M_0 = np.where((history_count[:,t-1]-history_count[:,changepoint_location])==0)[0]
        if len(M_0) == 0:
            sample_means = (history_reward[:,t-1]-history_reward[:,changepoint_location])/(history_count[:,t-1]-history_count[:,changepoint_location])
            a_t = np.argmax(np.abs(sample_means))
            history_sample_means[:,t]=sample_means
        else:
            a_t = np.random.choice(M_0)
    x_t = data[a_t,t]
    
    n[a_t] = n[a_t] + 1
    for m in range(M):
        history_count[m,t] = n[m]
    S[a_t] = S[a_t] + x_t
    for m in range(M):
        history_reward[m,t] = S[m]
    k = len(quadratics_positive[a_t])
    quadratic_add = [n[a_t],S[a_t],np.inf,t+1]
    i = k
    while (2*(quadratic_add[1]-quadratics_positive[a_t][i-1][1])-(quadratic_add[0]-quadratics_positive[a_t][i-1][0])*quadratics_positive[a_t][i-1][2])<=0 and i>=1:
        i = i-1
    quadratic_add[2] = max(0,2*(quadratic_add[0]-quadratics_positive[a_t][i-1][0])/(quadratic_add[2]-quadratics_positive[a_t][i-1][2]))
    quadratics_positive[a_t] = quadratics_positive[a_t][:i].copy()
    quadratics_positive[a_t].append(tuple(quadratic_add))

    # Now negative update
    k = len(quadratics_negative[a_t])
    quadratic_add = [n[a_t],S[a_t],-np.inf,t+1]
    i = k
    while (2*(quadratic_add[1]-quadratics_negative[a_t][i-1][1])-(quadratic_add[0]-quadratics_negative[a_t][i-1][0])*quadratics_negative[a_t][i-1][2])>=0 and i>=1:
        i = i-1
    quadratic_add[2] = min(0,2*(quadratic_add[0]-quadratics_negative[a_t][i-1][0])/(quadratic_add[2]-quadratics_negative[a_t][i-1][2]))
    quadratics_negative[a_t] = quadratics_negative[a_t][:i].copy()
    quadratics_negative[a_t].append(tuple(quadratic_add))
    
    changepoint_location, stream_estimate, best_glr = changepoint_estimator(quadratics_positive,quadratics_negative,S,n,M)
    changepoint_history.append(changepoint_location)
    stream_mle_history.append(stream_estimate)
    if best_glr>=threshold:
        print(f'stopping time: {t+1}')
        print(best_glr)
        break

stopping time: 50763
200.06155471071068


array([-0.09829165])

In [45]:
(35618-10000)/0.9

28464.444444444445